# Labour Data Generator
.csv file is from previous semester's synthetic parts generator

In [ ]:
import pandas as pd

# Assume df_cars_real is your existing DataFrame with car lifecycle data
# For demonstration, here's a minimal sample (in practice, use your full dataset):
df_cars_real = pd.read_csv("/content/all_parts.csv")

# Define a mapping from manufacturer/model to a car category
car_category_mapping = {
    "Ford": {
         "Focus": "Sedan",
         "Fusion": "Sedan",
         "Mustang": "Coupe",
         "Explorer": "SUV",
         "F-150": "Truck",
         "Escape": "SUV",
         "Edge": "SUV",
         "Ranger": "Truck"
    },
    "Toyota": {
         "Camry": "Sedan",
         "Corolla": "Sedan",
         "Prius": "Hatchback",
         "RAV4": "SUV",
         "Highlander": "SUV",
         "Tacoma": "Truck",
         "Tundra": "Truck",
         "Avalon": "Sedan"
    },
    "BMW": {
         "3 Series": "Sedan",
         "5 Series": "Sedan",
         "7 Series": "Sedan",
         "X5": "SUV",
         "X3": "SUV",
         "M3": "Coupe",
         "Z4": "Convertible",
         "X6": "SUV"
    },
    "Mercedes-Benz": {
         "C-Class": "Sedan",
         "E-Class": "Sedan",
         "S-Class": "Sedan",
         "GLC": "SUV",
         "GLE": "SUV",
         "A-Class": "Hatchback",
         "CLA": "Sedan",
         "G-Wagon": "SUV"
    }
}

# Define standard labour hours per car category
labour_hours_mapping = {
    "Sedan": 2.0,
    "SUV": 2.5,
    "Truck": 3.0,
    "Coupe": 1.8,
    "Convertible": 2.2,
    "Hatchback": 2.0
}

# Generate labour data based on the existing car lifecycle dataset
labour_data = []
for _, row in df_cars_real.iterrows():
    manufacturer = row["Manufacturer"]
    model = row["Model"]
    year = row["Year"]
    # Look up the car category; default to 'Sedan' if not found
    car_category = car_category_mapping.get(manufacturer, {}).get(model, "Sedan")
    hours = labour_hours_mapping.get(car_category, 2.0)
    labour_data.append({
         "Year": year,
         "Manufacturer": manufacturer,
         "Model": model,
         "Car Category": car_category,
         "Labour Hours": hours
    })

# Convert the labour data into a DataFrame
labour_df = pd.DataFrame(labour_data)

# Export the labour data to CSV (if needed)
labour_df.to_csv("labour_data.csv", index=False)

# Verify the structure
print(labour_df.head())

   Year Manufacturer   Model Car Category  Labour Hours
0  2000       Toyota  Avalon        Sedan           2.0
1  2000       Toyota  Avalon        Sedan           2.0
2  2000       Toyota  Avalon        Sedan           2.0
3  2000       Toyota  Avalon        Sedan           2.0
4  2000       Toyota  Avalon        Sedan           2.0


In [ ]:
import copy
import json

# Base labour mapping for repair tasks per parts category.
# (For brevity, only a few tasks per category are shown here. You can extend this to include all tasks.)
labour_mapping = {
    "Engine Components": [
        {
            "task": "Engine Diagnostics",
            "hours": 1.0,
            "description": (
                "Perform a complete diagnostic using OBD tools, visual inspection, and performance tests."
            ),
            "steps": [
                "Read fault codes using an OBD scanner",
                "Perform visual inspection of engine components",
                "Confirm issues with a test drive"
            ]
        },
        {
            "task": "Spark Plug Replacement",
            "hours": 1.2,
            "description": (
                "Remove worn spark plugs, check gap settings, and install new plugs with proper torque."
            ),
            "steps": [
                "Remove ignition coils",
                "Extract old spark plugs",
                "Install new spark plugs"
            ]
        }
    ],
    "Transmission System": [
        {
            "task": "Transmission Fluid Change",
            "hours": 2.0,
            "description": (
                "Drain old transmission fluid, replace the filter if necessary, and refill with new fluid."
            ),
            "steps": [
                "Lift the vehicle and locate the transmission pan",
                "Drain old fluid and remove filter",
                "Reassemble and refill with new fluid"
            ]
        },
        {
            "task": "Clutch Replacement",
            "hours": 3.0,
            "description": (
                "Remove the transmission, replace the clutch disc and pressure plate, then reassemble."
            ),
            "steps": [
                "Remove transmission",
                "Replace clutch assembly",
                "Reinstall transmission"
            ]
        }
    ],
    "Suspension and Steering System": [
        {
            "task": "Wheel Alignment",
            "hours": 1.0,
            "description": (
                "Adjust camber, caster, and toe settings to ensure proper tire wear and handling."
            ),
            "steps": [
                "Measure alignment settings",
                "Adjust settings using alignment equipment",
                "Test drive for confirmation"
            ]
        }
    ],
    "Braking System": [
        {
            "task": "Front Brake Reline",
            "hours": 1.0,
            "description": (
                "Resurface or replace the front brake rotors for effective braking."
            ),
            "steps": [
                "Remove the wheel and brake caliper",
                "Machine or replace the rotor",
                "Reassemble components"
            ]
        }
    ],
    "Electrical and Lighting System": [
        {
            "task": "Battery Replacement",
            "hours": 0.5,
            "description": (
                "Remove the old battery and install a new one with secure electrical connections."
            ),
            "steps": [
                "Disconnect battery terminals",
                "Remove old battery",
                "Install and reconnect new battery"
            ]
        }
    ],
    # ... (Add additional categories and tasks as needed)
}

# Dynamic vehicle-type labour hour multipliers.
# These multipliers adjust base labour hours based on vehicle type differences.
vehicle_type_modifiers = {
    "SUV": {
        "Engine Components": 4.1,
        "Transmission System": 3.05,
        "Suspension and Steering System": 2.2,
        "Braking System": 2.1,
        "Electrical and Lighting System": 1.0,
        # Add additional category multipliers if necessary
    },
    "Sedan": {
        "Engine Components": 4.0,
        "Transmission System": 3.0,
        "Suspension and Steering System": 2.0,
        "Braking System": 2.1,
        "Electrical and Lighting System": 1.0,
    },
    "Hatchback": {
        "Engine Components": 3.9,
        "Transmission System": 2.95,
        "Suspension and Steering System": 1.9,
        "Braking System": 1.95,
        "Electrical and Lighting System": 0.8,
    }
}

def get_dynamic_labour_mapping(vehicle_type, base_mapping):
    """
    Given a vehicle type (SUV, Sedan, Hatchback) and the base labour mapping,
    return a new mapping with labour hours adjusted by the vehicle type multipliers.
    """
    modifiers = vehicle_type_modifiers.get(vehicle_type, {})
    dynamic_mapping = {}

    # Process each category and its tasks
    for category, tasks in base_mapping.items():
        # Get the multiplier for this category; default is 1.0 if not defined
        multiplier = modifiers.get(category, 1.0)
        dynamic_tasks = []
        for task in tasks:
            # Create a deep copy of the task to avoid modifying the original mapping
            dynamic_task = copy.deepcopy(task)
            dynamic_task["hours"] = round(dynamic_task["hours"] * multiplier, 2)
            dynamic_tasks.append(dynamic_task)
        dynamic_mapping[category] = dynamic_tasks

    return dynamic_mapping

# Example usage:
dynamic_suv_labour = get_dynamic_labour_mapping("SUV", labour_mapping)
dynamic_sedan_labour = get_dynamic_labour_mapping("Sedan", labour_mapping)
dynamic_hatchback_labour = get_dynamic_labour_mapping("Hatchback", labour_mapping)

# Output the dynamic labour mappings (formatted as JSON for readability)
print("Dynamic Labour Mapping for SUVs:")
print(json.dumps(dynamic_suv_labour, indent=2))

print("\nDynamic Labour Mapping for Sedans:")
print(json.dumps(dynamic_sedan_labour, indent=2))

print("\nDynamic Labour Mapping for Hatchbacks:")
print(json.dumps(dynamic_hatchback_labour, indent=2))


Dynamic Labour Mapping for SUVs:
{
  "Engine Components": [
    {
      "task": "Engine Diagnostics",
      "hours": 4.1,
      "description": "Perform a complete diagnostic using OBD tools, visual inspection, and performance tests.",
      "steps": [
        "Read fault codes using an OBD scanner",
        "Perform visual inspection of engine components",
        "Confirm issues with a test drive"
      ]
    },
    {
      "task": "Spark Plug Replacement",
      "hours": 4.92,
      "description": "Remove worn spark plugs, check gap settings, and install new plugs with proper torque.",
      "steps": [
        "Remove ignition coils",
        "Extract old spark plugs",
        "Install new spark plugs"
      ]
    }
  ],
  "Transmission System": [
    {
      "task": "Transmission Fluid Change",
      "hours": 6.1,
      "description": "Drain old transmission fluid, replace the filter if necessary, and refill with new fluid.",
      "steps": [
        "Lift the vehicle and locate the

# Hyper-realistic
[Chat-GPT](https://chatgpt.com/share/67b65b18-71d4-800f-a883-5e3fd8e7c53d)



##

In [ ]:
import random
import pandas as pd
from dataclasses import dataclass, field
from typing import List, Optional

# ========================
# Service Code Definitions
# ========================

@dataclass
class Service:
    name: str
    labour_hours: float
    description: str

@dataclass
class ServiceCategory:
    category: str
    services: List[Service] = field(default_factory=list)
    note: Optional[str] = None

    def total_labour_hours(self) -> float:
        return sum(service.labour_hours for service in self.services)

def create_services() -> List[ServiceCategory]:
    # Engine Components
    engine_components = ServiceCategory(
        category="Engine Components",
        services=[
            Service(
                name="Engine Diagnostics",
                labour_hours=1.0,
                description=(
                    "Conduct a comprehensive assessment of the engine’s condition using advanced onboard diagnostics and visual inspection."
                )
            ),
            Service(
                name="Spark Plug Replacement",
                labour_hours=1.2,
                description=(
                    "Remove outdated spark plugs and install new ones after verifying proper gap settings to restore optimal ignition performance."
                )
            ),
            Service(
                name="Timing Belt Replacement",
                labour_hours=3.5,
                description=(
                    "Remove engine covers and ancillary components, align timing marks, remove the worn belt, and install a new one with precise tensioner adjustments."
                )
            ),
            Service(
                name="Cylinder Head Gasket Replacement",
                labour_hours=4.0,
                description=(
                    "Disassemble essential components to access the cylinder head, remove the deteriorated gasket, and prepare mating surfaces for a new gasket."
                )
            ),
            Service(
                name="Engine Rebuild",
                labour_hours=10.0,
                description=(
                    "Perform a complete teardown and reassembly of the engine, cleaning, inspecting, and replacing internal components as necessary."
                )
            )
        ]
    )

    # Transmission System
    transmission_system = ServiceCategory(
        category="Transmission System",
        services=[
            Service(
                name="Transmission Fluid Change",
                labour_hours=2.0,
                description=(
                    "Drain existing transmission fluid and replace it with the manufacturer-specified fluid, including a fluid filter change if applicable."
                )
            ),
            Service(
                name="Transmission Filter Replacement",
                labour_hours=1.5,
                description=(
                    "Access the transmission, replace the old filter with a new one and a proper gasket to maintain clean fluid flow."
                )
            ),
            Service(
                name="Clutch Replacement",
                labour_hours=3.0,
                description=(
                    "For manual transmissions, remove the transmission assembly to replace the worn clutch disc and pressure plate for smooth shifting."
                )
            ),
            Service(
                name="Transmission Diagnostics",
                labour_hours=1.5,
                description=(
                    "Utilize advanced diagnostic equipment to evaluate the transmission’s performance and pinpoint issues such as slippage."
                )
            ),
            Service(
                name="Transmission Rebuild",
                labour_hours=8.0,
                description=(
                    "Disassemble the transmission, inspect and replace worn components, and reassemble with exact alignment."
                )
            )
        ]
    )

    # Suspension and Steering System (with mandatory wheel alignment)
    suspension_steering = ServiceCategory(
        category="Suspension and Steering System",
        note="All suspension and steering repairs require a mandatory wheel alignment.",
        services=[
            Service(
                name="Suspension Inspection",
                labour_hours=0.5,
                description="Inspect shocks, struts, springs, and bushings for wear and damage."
            ),
            Service(
                name="Shock Absorber Replacement",
                labour_hours=1.5,
                description="Replace worn shock absorbers to restore optimal ride quality and handling."
            ),
            Service(
                name="Steering Rack Replacement",
                labour_hours=3.0,
                description="Remove and install a new steering rack with proper calibration."
            ),
            Service(
                name="Wheel Alignment",
                labour_hours=1.0,
                description="Adjust camber, caster, and toe to ensure optimal tire wear and handling."
            ),
            Service(
                name="Control Arm Replacement",
                labour_hours=2.0,
                description="Replace worn control arms and bushings to restore suspension geometry."
            )
        ]
    )

    # Braking System
    braking_system = ServiceCategory(
        category="Braking System",
        services=[
            Service(
                name="Front Brake Reline",
                labour_hours=1.0,
                description="Resurface or replace front brake rotors to ensure consistent braking performance."
            ),
            Service(
                name="Rear Brake Reline",
                labour_hours=1.0,
                description="Resurface or replace rear brake rotors to restore proper braking function."
            ),
            Service(
                name="Brake Bleeding",
                labour_hours=0.5,
                description="Remove trapped air from brake lines to restore optimal hydraulic pressure."
            ),
            Service(
                name="Brake Pad Replacement",
                labour_hours=1.5,
                description="Replace worn brake pads and inspect related components for balanced braking."
            ),
            Service(
                name="Caliper Service/Replacement",
                labour_hours=1.2,
                description="Service or replace brake calipers to correct sticking or leakage issues."
            )
        ]
    )

    # Additional categories (Electrical, Fuel, Cooling, Exhaust, Drivetrain, Body/Interior, HVAC, Safety, Signaling)
    # can be defined similarly. For brevity, they are omitted here but follow the same structure.

    return [engine_components, transmission_system, suspension_steering, braking_system]

# Define labour multipliers for car categories.
labour_multiplier = {
    "Sedan/Hatchback": 1.0,
    "SUV": 1.2,
    "Pickup Trucks": 1.3
}

def generate_services_for_model(car_info: dict, service_categories: List[ServiceCategory]) -> List[dict]:
    """
    Generate a list of service records for a given car. Each service's labour hours are
    adjusted based on the car category.
    """
    services_data = []
    multiplier = labour_multiplier.get(car_info["Car Category"], 1.0)
    for category in service_categories:
        for service in category.services:
            adjusted_labour = round(service.labour_hours * multiplier, 2)
            record = {
                "Year": car_info["Year"],
                "Manufacturer": car_info["Manufacturer"],
                "Model": car_info["Model"],
                "Engine Size (L)": car_info["Engine Size (L)"],
                "Transmission": car_info["Transmission"],
                "Car Category": car_info["Car Category"],
                "Service Category": category.category,
                "Service Name": service.name,
                "Labour Hours": adjusted_labour,
                "Description": service.description
            }
            services_data.append(record)
    return services_data

# ========================
# Parts & Fluids Generation Code (Modified)
# ========================

# Define parts categories and breakdown
car_parts = {
    "Engine Components": ["Engine Block", "Pistons", "Crankshaft", "Camshaft", "Valves", "Timing Belt", "Ignition System"],
    "Transmission System": ["Transmission", "Clutch Assembly", "Gearbox", "Differential"],
    "Suspension and Steering System": ["Steering Components", "Suspension Components"],
    "Braking System": ["Disc Brakes", "Drum Brakes", "Hydraulic System"],
    "Electrical and Lighting System": ["Battery", "Headlights", "Tail Lights"],
    "Fuel System": ["Fuel Tank", "Fuel Pump", "Fuel Filter"],
    "Cooling System": ["Radiator", "Water Pump"],
    "Exhaust System": ["Exhaust Manifold", "Catalytic Converter", "Muffler"],
    "Drivetrain System": ["Axles", "CV Joints"],
    "Body and Interior Components": ["Chassis/Frame", "Doors", "Seats"],
    "HVAC System": ["Compressor", "Blower Motor"],
    "Safety and Security System": ["Airbags", "Seat Belts", "Parking Sensors"],
    "Signaling System": ["Turn Signals", "Hazard Lights"]
}

# Define part manufacturers and pricing tiers
part_manufacturers = {
    "Pro-Series OE": "PE",
    "Pro-Series OE Plus": "PS",
    "Brembo": "BO",
    "Centric": "CC"
}

# Base prices for parts categories
base_prices = {
    "Engine Components": 500,
    "Transmission System": 800,
    "Suspension and Steering System": 300,
    "Braking System": 200,
    "Electrical and Lighting System": 100,
    "Fuel System": 250,
    "Cooling System": 400,
    "Exhaust System": 350,
    "Drivetrain System": 600,
    "Body and Interior Components": 700,
    "HVAC System": 450,
    "Safety and Security System": 200,
    "Signaling System": 50
}

# Define fluid types and manufacturers
fluid_manufacturers = {
    "Castrol": "Cheap",
    "Pennzoil": "Affordable",
    "Mobil1": "Expensive"
}

fluid_types = {
    "Engine Oil": ["5W-20", "0W-20", "5W-30"],
    "Brake Fluid": ["DOT-3", "DOT-4"],
    "Manual Transmission Fluid": ["MTF-75W-90"],
    "Automatic Transmission Fluid": ["ATF-Dexron", "ATF-Mercon"],
    "Coolant/Antifreeze": ["Pre-mixed Coolant", "Concentrated Coolant"],
    "Power Steering Fluid": ["Standard", "Synthetic"],
    "Gear Oil": ["75W-90", "80W-90"],
    "Grease": ["Lithium Grease", "Synthetic Grease"],
    "Transfer Case Fluid": ["ATF 4", "Full Synthetic"],
    "Refrigerant": ["R-134a", "R-1234yf"]
}

# Dictionaries to track generated numbers and prices for consistency
generated_part_numbers = {}
generated_fluid_numbers = {}
generated_prices = {}

def generate_part_number(part_name, manufacturer_code):
    part_key = f"{manufacturer_code}_{part_name}"
    if part_key in generated_part_numbers:
        return generated_part_numbers[part_key]
    else:
        part_number_suffix = random.randint(1000, 9999)
        generated_part_numbers[part_key] = part_number_suffix
        return part_number_suffix

def generate_fluid_number(fluid_name, fluid_subtype, fluid_manufacturer):
    fluid_key = f"{fluid_manufacturer}_{fluid_name}_{fluid_subtype}"
    if fluid_key in generated_fluid_numbers:
        return generated_fluid_numbers[fluid_key]
    else:
        fluid_number_suffix = random.randint(1000, 9999)
        generated_fluid_numbers[fluid_key] = fluid_number_suffix
        return fluid_number_suffix

def generate_parts_for_model(car_info: dict) -> List[dict]:
    parts_data = []
    for category, parts in car_parts.items():
        for part in parts:
            base_price = base_prices.get(category, 100)
            for part_manufacturer, manufacturer_code in part_manufacturers.items():
                part_number_suffix = generate_part_number(part, manufacturer_code)
                part_number = f"{manufacturer_code}{part[:3].upper()}{part_number_suffix}"
                # Ensure consistent pricing for the same part number
                part_key = f"{part_number}"
                if part_key not in generated_prices:
                    price = round(base_price * random.uniform(0.9, 1.1), 2)
                    generated_prices[part_key] = price
                else:
                    price = generated_prices[part_key]

                record = {
                    "Year": car_info["Year"],
                    "Manufacturer": car_info["Manufacturer"],
                    "Model": car_info["Model"],
                    "Engine Size (L)": car_info["Engine Size (L)"],
                    "Transmission": car_info["Transmission"],
                    "Car Category": car_info["Car Category"],
                    "Part Category": category,
                    "Part Name": part,
                    "Part Manufacturer": part_manufacturer,
                    "Part Number": part_number,
                    "Price ($)": price
                }
                parts_data.append(record)
    return parts_data

def generate_fluids_for_model(car_info: dict) -> List[dict]:
    fluid_data = []
    transmission = car_info["Transmission"]
    year = car_info["Year"]

    # Determine the correct transmission fluid type
    if transmission == "Manual":
        transmission_fluid_type = random.choice(fluid_types["Manual Transmission Fluid"])
    else:
        transmission_fluid_type = random.choice(fluid_types["Automatic Transmission Fluid"])

    # Determine refrigerant based on the year
    if year < 2012:
        refrigerant_type = "R-134a"
    elif year > 2019:
        refrigerant_type = "R-1234yf"
    else:
        refrigerant_type = random.choice(fluid_types["Refrigerant"])

    fluids_to_assign = {
        "Engine Oil": random.choice(fluid_types["Engine Oil"]),
        "Brake Fluid": random.choice(fluid_types["Brake Fluid"]),
        "Transmission Fluid": transmission_fluid_type,
        "Coolant/Antifreeze": random.choice(fluid_types["Coolant/Antifreeze"]),
        "Power Steering Fluid": random.choice(fluid_types["Power Steering Fluid"]),
        "Gear Oil": random.choice(fluid_types["Gear Oil"]),
        "Grease": random.choice(fluid_types["Grease"]),
        "Transfer Case Fluid": random.choice(fluid_types["Transfer Case Fluid"]),
        "Refrigerant": refrigerant_type
    }

    for fluid_name, fluid_subtype in fluids_to_assign.items():
        for fluid_manufacturer, tier in fluid_manufacturers.items():
            fluid_number_suffix = generate_fluid_number(fluid_name, fluid_subtype, fluid_manufacturer)
            fluid_number = f"{fluid_manufacturer[:2].upper()}-{fluid_name[:3].upper()}{fluid_number_suffix}"
            fluid_key = f"{fluid_number}"
            if fluid_key not in generated_prices:
                fluid_price = round(random.uniform(30, 80), 2)
                generated_prices[fluid_key] = fluid_price
            else:
                fluid_price = generated_prices[fluid_key]

            record = {
                "Year": car_info["Year"],
                "Manufacturer": car_info["Manufacturer"],
                "Model": car_info["Model"],
                "Engine Size (L)": car_info["Engine Size (L)"],
                "Transmission": car_info["Transmission"],
                "Car Category": car_info["Car Category"],
                "Fluid Type": fluid_name,
                "Fluid Subtype": fluid_subtype,
                "Fluid Manufacturer": fluid_manufacturer,
                "Fluid Number": fluid_number,
                "Price ($)": fluid_price
            }
            fluid_data.append(record)
    return fluid_data

# ========================
# Car Lifecycle Data Generation
# ========================

# Define realistic car manufacturers and models
real_manufacturers = {
    "Ford": ["Focus", "Mustang", "Explorer", "Fusion", "F-150", "Escape", "Edge", "Ranger"],
    "Toyota": ["Camry", "Corolla", "Prius", "RAV4", "Highlander", "Tacoma", "Tundra", "Avalon"],
    "BMW": ["3 Series", "5 Series", "7 Series", "X5", "X3", "M3", "Z4", "X6"],
    "Mercedes-Benz": ["C-Class", "E-Class", "S-Class", "GLC", "GLE", "A-Class", "CLA", "G-Wagon"]
}

engine_sizes = [1.8, 2.4, 3.2, 5.7]
transmissions = ["Manual", "Automatic"]

# Define car categories and assign one to each car.
car_categories = ["Sedan/Hatchback", "SUV", "Pickup Trucks"]

car_data_real = []
start_year = 2000
end_year = 2025
model_change_rate = 0.25  # 25% model replacement rate per year

for year in range(start_year, end_year + 1):
    for manufacturer, models in real_manufacturers.items():
        models_for_year = models[:]
        num_models_to_replace = int(len(models_for_year) * model_change_rate)
        if num_models_to_replace > 0:
            models_for_year = models_for_year[num_models_to_replace:]
        num_new_models = max(8 - len(models_for_year), 0)
        available_models = list(set(models) - set(models_for_year))
        new_models = random.sample(available_models, min(num_new_models, len(available_models)))
        models_for_year.extend(new_models)
        for model in models_for_year:
            for engine_size in engine_sizes:
                car_record = {
                    "Year": year,
                    "Manufacturer": manufacturer,
                    "Model": model,
                    "Engine Size (L)": engine_size,
                    "Transmission": random.choice(transmissions),
                    "Car Category": random.choice(car_categories)  # Randomly assign a car category
                }
                car_data_real.append(car_record)

df_cars_real = pd.DataFrame(car_data_real)

# ========================
# Combine Parts, Fluids, and Services Data
# ========================

all_parts_fluids = []
all_services = []

# Pre-create the service categories list (from our service definitions)
service_categories = create_services()

for _, row in df_cars_real.iterrows():
    car_info = row.to_dict()
    parts_data = generate_parts_for_model(car_info)
    fluids_data = generate_fluids_for_model(car_info)
    services_data = generate_services_for_model(car_info, service_categories)

    all_parts_fluids.extend(parts_data)
    all_parts_fluids.extend(fluids_data)
    all_services.extend(services_data)

# Convert to DataFrames
df_parts_fluids = pd.DataFrame(all_parts_fluids)
df_services = pd.DataFrame(all_services)

# Reorder columns for parts/fluids (ensure Price is the last column)
ordered_columns_parts = [
    "Year", "Manufacturer", "Model", "Engine Size (L)", "Transmission", "Car Category",
    "Part Category", "Part Name", "Part Manufacturer", "Part Number",
    "Fluid Type", "Fluid Subtype", "Fluid Manufacturer", "Fluid Number", "Price ($)"
]
# Some rows (parts vs fluids) have different columns; we’ll let pandas handle missing columns.

# Save the datasets to CSV files
df_parts_fluids.to_csv("all_parts_fluids.csv", index=False)
df_services.to_csv("services_data.csv", index=False)

# Print summary info
print("Parts and Fluids DataFrame Info:")
print(df_parts_fluids.info())
print("\nServices DataFrame Info:")
print(df_services.info())


Parts and Fluids DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609024 entries, 0 to 609023
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Year                609024 non-null  int64  
 1   Manufacturer        609024 non-null  object 
 2   Model               609024 non-null  object 
 3   Engine Size (L)     609024 non-null  float64
 4   Transmission        609024 non-null  object 
 5   Car Category        609024 non-null  object 
 6   Part Category       519168 non-null  object 
 7   Part Name           519168 non-null  object 
 8   Part Manufacturer   519168 non-null  object 
 9   Part Number         519168 non-null  object 
 10  Price ($)           609024 non-null  float64
 11  Fluid Type          89856 non-null   object 
 12  Fluid Subtype       89856 non-null   object 
 13  Fluid Manufacturer  89856 non-null   object 
 14  Fluid Number        89856 non-null   object 
dtypes

In [ ]:
df_services.head()

,Year,Manufacturer,Model,Engine Size (L),Transmission,Car Category,Service Category,Service Name,Labour Hours,Description
0,2000,Ford,Explorer,1.8,Manual,Sedan/Hatchback,Engine Components,Engine Diagnostics,1.0,Conduct a comprehensive assessment of the engi...
1,2000,Ford,Explorer,1.8,Manual,Sedan/Hatchback,Engine Components,Spark Plug Replacement,1.2,Remove outdated spark plugs and install new on...
2,2000,Ford,Explorer,1.8,Manual,Sedan/Hatchback,Engine Components,Timing Belt Replacement,3.5,"Remove engine covers and ancillary components,..."
3,2000,Ford,Explorer,1.8,Manual,Sedan/Hatchback,Engine Components,Cylinder Head Gasket Replacement,4.0,Disassemble essential components to access the...
4,2000,Ford,Explorer,1.8,Manual,Sedan/Hatchback,Engine Components,Engine Rebuild,10.0,Perform a complete teardown and reassembly of ...
